# Distribution Parameters Estimation

In [215]:
from scipy.stats import beta, bernoulli
import numpy as np
rng = np.random.default_rng()

Use real probability of $p_1=0.9$ for each bit to be $b_i=1$ as an example (non-uniform prior). The

Use $N=50$ samples for estimating in each step.

In [216]:
p1 = 0.9
N = 50
b_i = bernoulli(p1)

## MLE
We attempt an MLE estimate.

We realize a 100 sets with N realization in each. Then make 100 estimates of $p$.

Using the MLE, the estimator is simply the sample mean: $\hat{p}_1=\frac{1}{N}\sum_{i=1}^N x_i$

Since $N$ is small the MLE preforms poorly.

In [217]:
l = []
for _ in range(100):
    x = b_i.rvs(size=N, random_state=rng)
    l.append(np.mean(x))
p1_hat = np.array(l)
estimation_error = p1_hat-p1
max_error = max(np.abs(estimation_error))
MSE = np.power(np.abs(estimation_error), 2).mean()

The maximal estimation error and MSE are:

In [218]:
max_error

0.09999999999999998

In [219]:
MSE

0.0018399999999999994

In [220]:
import plotly.express as px
import pandas as pd
data = pd.DataFrame({"error":np.abs(estimation_error)})
fig = px.scatter(data, y="error")
fig.update_traces(marker_size=10)
fig.show()

## Bayesian / MAP Estimate

We can reduce estimation error if we acount for some uncertainty in our estimate $\hat{p}_1$. We therfore estimate the *distribution* of $p_1$. If we have no priors the most naive guess is simply $p_1\sim U(0,1)$.

Instead, we can make posterior estimates using both the priors and observations as shown [here](https://web.stanford.edu/class/archive/cs/cs109/cs109.1192/reader/8%20Probability%20as%20a%20Variable.pdf).

As shown in the link, the posterior probability descrbing is distributed as $p_1\sim Beta(a,b)$. 

### Bayesian Estimation using Conjugate Priors

Using samples to improve estimates is simple, since both the prior and posterior are Beta distributed.

We assume priors $a,b$ such that out of $a+b-2$ *hypothetical* samples, $a-1$ will be `1`, and $b-1$ will `0`.

If we then take actaul samples, and observe $n$ values of `1` and $m$ observations of `0`, the posterior distribtion is $p_1\vert X \sim Beta(a+n,b+m)$.

This process can be repeated iteratively to refine the estimate.

In [221]:
l = []
# initial prior is uniform
a = 1 
b = 1
for _ in range(100):
    x = b_i.rvs(size=N, random_state=rng)
    n = np.sum(x)
    a += n
    b += N-n
    l.append((a-1)/(a+b-2))  # estimate using mode see wiki: https://en.wikipedia.org/wiki/Beta_distribution 
rv = beta(a,b)
m,v = rv.stats(moments='mv')
p1_hat = np.array(l)  
estimation_error = p1_hat-p1
max_error = max(np.abs(estimation_error))
MSE = np.power(np.abs(estimation_error), 2).mean()

In [222]:
p1_hat[-1]

0.9

In [223]:
max_error

0.020000000000000018

In [224]:
import plotly.express as px
import pandas as pd
data = pd.DataFrame({"error":np.abs(estimation_error)})
fig = px.line(data, y="error")
# fig.update_traces(marker_size=10)
fig.show()

# Adding NoiseWe attempt an MLE estimate.

We add noise to the measurements. The noise is also Bernoulli distributed as is flips a bit with probability $f$ such that: $W\sim B(f)$.

Assume $f=0.05$.


In [225]:
f = 0.05
noise = bernoulli(f)

## MLE

In [226]:
l = []
for _ in range(100):
    x = b_i.rvs(size=N, random_state=rng)
    w = noise.rvs(size=N, random_state=rng)
    x = np.mod(x + w, 2)
    l.append(np.mean(x))
p1_hat = np.array(l)
estimation_error = p1_hat-p1
max_error = max(np.abs(estimation_error))
MSE = np.power(np.abs(estimation_error), 2).mean()

The maximal estimation error and MSE are:

In [227]:
max_error

0.20000000000000007

In [228]:
MSE

0.003532000000000003

In [229]:
import plotly.express as px
import pandas as pd
data = pd.DataFrame({"error":np.abs(estimation_error)})
fig = px.scatter(data, y="error")
fig.update_traces(marker_size=10)
fig.show()

## Bayesian estimate ignoring noise

In [230]:
l = []
# initial prior is uniform
a = 1
b = 1
for _ in range(100):
    x = b_i.rvs(size=N, random_state=rng)
    w = noise.rvs(size=N, random_state=rng)
    x = np.mod(x + w, 2)
    n = np.sum(x)
    a += n
    b += N-n
    l.append((a-1)/(a+b-2))  # estimate using mode see wiki: https://en.wikipedia.org/wiki/Beta_distribution
rv = beta(a,b)
m,v = rv.stats(moments='mv')
p1_hat = np.array(l)
estimation_error = p1_hat-p1
max_error = max(np.abs(estimation_error))
MSE = np.power(np.abs(estimation_error), 2).mean()

In [231]:
import plotly.express as px
import pandas as pd
data = pd.DataFrame({"error":np.abs(estimation_error)})
fig = px.line(data, y="error")
fig.show()

Acounting for noise, the noisy bit is distributed Bernoulli with $X=b\oplus w \sim B(v)$, with $v=p_1-f(2p_1-1)$

In [232]:
v = []
# initial prior is uniform
a = 1
b = 1
for _ in range(100):
    x = b_i.rvs(size=N, random_state=rng)
    w = noise.rvs(size=N, random_state=rng)
    x = np.mod(x + w, 2)
    n = np.sum(x)
    a += n
    b += N-n
    v.append((a-1)/(a+b-2))  # estimate using mode see wiki: https://en.wikipedia.org/wiki/Beta_distribution
v = np.array(v)
p1_hat = (v-f)/(1-2*f)
estimation_error = p1_hat-p1
max_error = max(np.abs(estimation_error))
MSE = np.power(np.abs(estimation_error), 2).mean()

In [233]:
import plotly.express as px
import pandas as pd
data = pd.DataFrame({"error":np.abs(estimation_error)})
fig = px.line(data, y="error")
fig.show()